In [1]:
from widerface_evaluate.evaluation import get_preds

In [2]:
%%bash
cd widerface_evaluate
python setup.py build_ext --inplace

Compiling box_overlaps.pyx because it changed.
[1/1] Cythonizing box_overlaps.pyx
running build_ext
building 'bbox' extension
creating build
creating build/temp.linux-x86_64-3.7
gcc -pthread -B /home/chec/anaconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/chec/anaconda3/lib/python3.7/site-packages/numpy/core/include -I/home/chec/anaconda3/include/python3.7m -c box_overlaps.c -o build/temp.linux-x86_64-3.7/box_overlaps.o
gcc -pthread -shared -B /home/chec/anaconda3/compiler_compat -L/home/chec/anaconda3/lib -Wl,-rpath=/home/chec/anaconda3/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.7/box_overlaps.o -o /vc_data/chec/Pytorch_Retinaface/widerface_evaluate/bbox.cpython-37m-x86_64-linux-gnu.so


In file included from /home/chec/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832:0,
                 from /home/chec/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/chec/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from box_overlaps.c:617:
/home/chec/anaconda3/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
 #warning "Using deprecated NumPy API, disable it with " \
  ^~~~~~~
/home/chec/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /vc_data/chec/Pytorch_Retinaface/widerface_evaluate/box_overlaps.pyx
  tree = Parsing.p_module(s, pxd, full_m

In [5]:
preds = get_preds('./widerface_evaluate/widerface_txt/')

Reading Predictions : 100%|██████████| 61/61 [00:07<00:00,  7.85it/s]


In [6]:
type(preds)

dict

In [7]:
list(preds.keys())

['15--Stock_Market',
 '46--Jockey',
 '47--Matador_Bullfighter',
 '11--Meeting',
 '26--Soldier_Drilling',
 '39--Ice_Skating',
 '29--Students_Schoolkids',
 '57--Angler',
 '18--Concerts',
 '45--Balloonist',
 '24--Soldier_Firing',
 '31--Waiter_Waitress',
 '41--Swimming',
 '0--Parade',
 '40--Gymnastics',
 '2--Demonstration',
 '43--Row_Boat',
 '36--Football',
 '27--Spa',
 '12--Group',
 '34--Baseball',
 '6--Funeral',
 '51--Dresses',
 '10--People_Marching',
 '56--Voter',
 '42--Car_Racing',
 '61--Street_Battle',
 '9--Press_Conference',
 '14--Traffic',
 '50--Celebration_Or_Party',
 '38--Tennis',
 '48--Parachutist_Paratrooper',
 '5--Car_Accident',
 '35--Basketball',
 '19--Couple',
 '16--Award_Ceremony',
 '8--Election_Campain',
 '55--Sports_Coach_Trainer',
 '49--Greeting',
 '22--Picnic',
 '21--Festival',
 '20--Family_Group',
 '54--Rescue',
 '7--Cheering',
 '25--Soldier_Patrol',
 '32--Worker_Laborer',
 '28--Sports_Fan',
 '44--Aerobics',
 '30--Surgeons',
 '52--Photographers',
 '53--Raid',
 '3--Riot'

In [8]:
type(preds['16--Award_Ceremony'])

dict

In [9]:
list(preds['16--Award_Ceremony'].keys())

['16_Award_Ceremony_Awards_Ceremony_16_512',
 '16_Award_Ceremony_Awards_Ceremony_16_239',
 '16_Award_Ceremony_Awards_Ceremony_16_495',
 '16_Award_Ceremony_Awards_Ceremony_16_591',
 '16_Award_Ceremony_Awards_Ceremony_16_56',
 '16_Award_Ceremony_Awards_Ceremony_16_195',
 '16_Award_Ceremony_Awards_Ceremony_16_490',
 '16_Award_Ceremony_Awards_Ceremony_16_4',
 '16_Award_Ceremony_Awards_Ceremony_16_422',
 '16_Award_Ceremony_Awards_Ceremony_16_135',
 '16_Award_Ceremony_Awards_Ceremony_16_546',
 '16_Award_Ceremony_Awards_Ceremony_16_566',
 '16_Award_Ceremony_Awards_Ceremony_16_569',
 '16_Award_Ceremony_Awards_Ceremony_16_25',
 '16_Award_Ceremony_Awards_Ceremony_16_143',
 '16_Award_Ceremony_Awards_Ceremony_16_361',
 '16_Award_Ceremony_Awards_Ceremony_16_64',
 '16_Award_Ceremony_Awards_Ceremony_16_447',
 '16_Award_Ceremony_Awards_Ceremony_16_524',
 '16_Award_Ceremony_Awards_Ceremony_16_752',
 '16_Award_Ceremony_Awards_Ceremony_16_338',
 '16_Award_Ceremony_Awards_Ceremony_16_392',
 '16_Award_Cere

In [10]:
preds['16--Award_Ceremony']['16_Award_Ceremony_Awards_Ceremony_16_25']

array([[ 5.0200000e+02,  9.4000000e+01,  3.3000000e+01,  4.4000000e+01,
         9.9878890e-01],
       [ 4.4000000e+02,  1.2200000e+02,  3.3000000e+01,  4.3000000e+01,
         9.9794906e-01],
       [ 8.4300000e+02,  1.3300000e+02,  3.8000000e+01,  4.7000000e+01,
         9.9721164e-01],
       [ 6.5000000e+02,  1.3700000e+02,  3.8000000e+01,  4.7000000e+01,
         9.9605995e-01],
       [ 5.6300000e+02,  1.0300000e+02,  4.0000000e+01,  5.4000000e+01,
         9.9570730e-01],
       [ 2.2500000e+02,  1.1800000e+02,  3.9000000e+01,  4.9000000e+01,
         9.9287620e-01],
       [ 7.4000000e+02,  1.4600000e+02,  4.2000000e+01,  5.5000000e+01,
         9.9030393e-01],
       [ 9.2900000e+02,  1.5300000e+02,  4.2000000e+01,  5.4000000e+01,
         9.8621200e-01],
       [ 1.4900000e+02,  9.6000000e+01,  3.9000000e+01,  5.4000000e+01,
         9.8590000e-01],
       [ 3.3200000e+02,  1.4800000e+02,  4.3000000e+01,  5.2000000e+01,
         9.8317873e-01],
       [ 6.0000000e+01,  1.020

In [11]:
preds['16--Award_Ceremony']['16_Award_Ceremony_Awards_Ceremony_16_25'].shape

(52, 5)

# Generate predictions

In [39]:
from __future__ import print_function
import os
import argparse
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from data import cfg_mnet, cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2
from models.retinaface import RetinaFace
from utils.box_utils import decode, decode_landm
from utils.timer import Timer


parser = argparse.ArgumentParser(description='Retinaface')
parser.add_argument('-m', '--trained_model', default='./weights/Resnet50_Final.pth',
                    type=str, help='Trained state_dict file path to open')
parser.add_argument('--network', default='resnet50', help='Backbone network mobile0.25 or resnet50')
parser.add_argument('--origin_size', default=True, type=str, help='Whether use origin image size to evaluate')
parser.add_argument('--save_folder', default='./widerface_evaluate/widerface_txt/', type=str, help='Dir to save txt results')
parser.add_argument('--cpu', action="store_true", default=False, help='Use cpu inference')
parser.add_argument('--dataset_folder', default='./data/widerface/val/images/', type=str, help='dataset path')
parser.add_argument('--confidence_threshold', default=0.01, type=float, help='confidence_threshold')
parser.add_argument('--top_k', default=5000, type=int, help='top_k')
parser.add_argument('--nms_threshold', default=0.4, type=float, help='nms_threshold')
parser.add_argument('--keep_top_k', default=750, type=int, help='keep_top_k')
parser.add_argument('-s', '--save_image', action="store_true", default=False, help='show detection results')
parser.add_argument('--vis_thres', default=0.5, type=float, help='visualization_threshold')
#args = parser.parse_args()

args = argparse.Namespace()
args.trained_model = './weights/mobilenet0.25_Final.pth'
args.network = 'mobile0.25'
args.origin_size = True
args.save_folder = './widerface_evaluate/widerface_txt/'
args.cpu = False
args.dataset_folder = './data/widerface/val/images/'
args.confidence_threshold = 0.01
args.top_k = 5000
args.nms_threshold = 0.4
args.keep_top_k = 750
args.save_image = False
args.vis_thres = 0.3


def check_keys(model, pretrained_state_dict):
    ckpt_keys = set(pretrained_state_dict.keys())
    model_keys = set(model.state_dict().keys())
    used_pretrained_keys = model_keys & ckpt_keys
    unused_pretrained_keys = ckpt_keys - model_keys
    missing_keys = model_keys - ckpt_keys
    print('Missing keys:{}'.format(len(missing_keys)))
    print('Unused checkpoint keys:{}'.format(len(unused_pretrained_keys)))
    print('Used keys:{}'.format(len(used_pretrained_keys)))
    assert len(used_pretrained_keys) > 0, 'load NONE from pretrained checkpoint'
    return True


def remove_prefix(state_dict, prefix):
    ''' Old style model is stored with all names of parameters sharing common prefix 'module.' '''
    print('remove prefix \'{}\''.format(prefix))
    f = lambda x: x.split(prefix, 1)[-1] if x.startswith(prefix) else x
    return {f(key): value for key, value in state_dict.items()}


def load_model(model, pretrained_path, load_to_cpu):
    print('Loading pretrained model from {}'.format(pretrained_path))
    if load_to_cpu:
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage)
    else:
        device = torch.cuda.current_device()
        pretrained_dict = torch.load(pretrained_path, map_location=lambda storage, loc: storage.cuda(device))
    if "state_dict" in pretrained_dict.keys():
        pretrained_dict = remove_prefix(pretrained_dict['state_dict'], 'module.')
    else:
        pretrained_dict = remove_prefix(pretrained_dict, 'module.')
    check_keys(model, pretrained_dict)
    model.load_state_dict(pretrained_dict, strict=False)
    return model




In [40]:
from collections import defaultdict
def predict():
    torch.set_grad_enabled(False)

    cfg = None
    if args.network == "mobile0.25":
        cfg = cfg_mnet
    elif args.network == "resnet50":
        cfg = cfg_re50
    # net and model
    net = RetinaFace(cfg=cfg, phase = 'test')
    net = load_model(net, args.trained_model, args.cpu)
    net.eval()
    print('Finished loading model!')
    #print(net)
    cudnn.benchmark = True
    device = torch.device("cpu" if args.cpu else "cuda")
    net = net.to(device)

    # testing dataset
    testset_folder = args.dataset_folder
    #testset_list = args.dataset_folder[:-7] + "wider_val.txt"
    testset_list = args.dataset_folder[:-7] + "wider_val_half.txt"

    with open(testset_list, 'r') as fr:
        test_dataset = fr.read().split()[:10]
    num_images = len(test_dataset)

    _t = {'forward_pass': Timer(), 'misc': Timer()}

    # testing begin
    results = defaultdict(dict)
    for i, img_name in enumerate(test_dataset):
        print('img_name:', img_name)
        image_path = testset_folder + img_name
        img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = np.float32(img_raw)

        # testing scale
        target_size = 1600
        max_size = 2150
        im_shape = img.shape
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])
        resize = float(target_size) / float(im_size_min)
        # prevent bigger axis from being more than max_size:
        if np.round(resize * im_size_max) > max_size:
            resize = float(max_size) / float(im_size_max)
        if args.origin_size:
            resize = 1

        if resize != 1:
            img = cv2.resize(img, None, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)
        im_height, im_width, _ = img.shape
        scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
        img -= (104, 117, 123)
        img = img.transpose(2, 0, 1)
        img = torch.from_numpy(img).unsqueeze(0)
        img = img.to(device)
        scale = scale.to(device)

        _t['forward_pass'].tic()
        loc, conf, landms = net(img)  # forward pass
        _t['forward_pass'].toc()
        _t['misc'].tic()
        priorbox = PriorBox(cfg, image_size=(im_height, im_width))
        priors = priorbox.forward()
        priors = priors.to(device)
        prior_data = priors.data
        boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
        boxes = boxes * scale / resize
        boxes = boxes.cpu().numpy()
        scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
        landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
        scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2]])
        scale1 = scale1.to(device)
        landms = landms * scale1 / resize
        landms = landms.cpu().numpy()

        # ignore low scores
        inds = np.where(scores > args.confidence_threshold)[0]
        boxes = boxes[inds]
        landms = landms[inds]
        scores = scores[inds]
        
        print('num dets before nms:', len(scores))

        # keep top-K before NMS
        order = scores.argsort()[::-1]
        # order = scores.argsort()[::-1][:args.top_k]
        boxes = boxes[order]
        landms = landms[order]
        scores = scores[order]

        # do NMS
        dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
        #keep = py_cpu_nms(dets, args.nms_threshold)
        # keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
        #dets = dets[keep, :]
        #landms = landms[keep]

        # keep top-K faster NMS
        # dets = dets[:args.keep_top_k, :]
        # landms = landms[:args.keep_top_k, :]

        #dets = np.concatenate((dets, landms), axis=1)
        _t['misc'].toc()
        
        event = img_name.split('/')[1]
        event_img = img_name.split('/')[2][:-4]

        #if event_img not in results[event]:
        dets[:, 2] = dets[:, 2] - dets[:, 0]
        dets[:, 3] = dets[:, 3] - dets[:, 1]
        print('num final dets:', len(dets))
        results[event][event_img] = dets[:, :5]
        #print('results:', results)
        print(min(dets[:, 4]))
    return results
            

        

In [41]:
results = predict()

Loading pretrained model from ./weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Finished loading model!
img_name: /30--Surgeons/30_Surgeons_Surgeons_30_819.jpg
num dets before nms: 106
num final dets: 106
0.0100878095
img_name: /61--Street_Battle/61_Street_Battle_streetfight_61_395.jpg
num dets before nms: 694
num final dets: 694
0.010005754
img_name: /56--Voter/56_Voter_peoplevoting_56_122.jpg
num dets before nms: 2924
num final dets: 2924
0.010009705
img_name: /21--Festival/21_Festival_Festival_21_462.jpg
num dets before nms: 168
num final dets: 168
0.010059146
img_name: /52--Photographers/52_Photographers_photographertakingphoto_52_125.jpg
num dets before nms: 203
num final dets: 203
0.010033634
img_name: /2--Demonstration/2_Demonstration_Political_Rally_2_137.jpg
num dets before nms: 1975
num final dets: 1975
0.010007412
img_name: /35--Basketball/35_Basketball_basketballgame_ball_35_681.jpg
num dets before nms: 1162
num 

In [21]:
results = predict()

Loading pretrained model from ./weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Finished loading model!
img_name: /30--Surgeons/30_Surgeons_Surgeons_30_819.jpg
num dets before nms: 106
num final dets: 8
img_name: /61--Street_Battle/61_Street_Battle_streetfight_61_395.jpg
num dets before nms: 694
num final dets: 127
img_name: /56--Voter/56_Voter_peoplevoting_56_122.jpg
num dets before nms: 2924
num final dets: 731
img_name: /21--Festival/21_Festival_Festival_21_462.jpg
num dets before nms: 168
num final dets: 42
img_name: /52--Photographers/52_Photographers_photographertakingphoto_52_125.jpg
num dets before nms: 203
num final dets: 31
img_name: /2--Demonstration/2_Demonstration_Political_Rally_2_137.jpg
num dets before nms: 1975
num final dets: 620
img_name: /35--Basketball/35_Basketball_basketballgame_ball_35_681.jpg
num dets before nms: 1162
num final dets: 292
img_name: /35--Basketball/35_Basketball_basketballgame_ball_35_

In [110]:
results_half = predict()

Loading pretrained model from ./weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Finished loading model!


In [25]:
isinstance(results, dict)

True

In [5]:
%load_ext autoreload

In [6]:
%autoreload 2

In [7]:
from widerface_evaluate.evaluation import evaluation

In [42]:
evaluation(results, './widerface_evaluate/ground_truth/')

Processing easy:  30%|██▉       | 18/61 [00:00<00:00, 179.33it/s]

pred: [[1.38822784e+01 3.71722168e+02 8.03925400e+01 1.23177612e+02
  9.99365211e-01]
 [1.38418617e+01 3.71103943e+02 7.88164978e+01 1.21154175e+02
  9.98752832e-01]
 [1.33732185e+01 3.71722931e+02 8.01895142e+01 1.21477539e+02
  9.98703003e-01]
 ...
 [1.16287659e+02 4.88374664e+02 3.65837402e+01 4.03504944e+01
  7.86295914e-06]
 [7.06868057e+01 3.16275513e+02 1.84920120e+01 2.49289551e+01
  6.93242600e-06]
 [1.33875153e+02 4.16106262e+02 3.51509094e+01 4.47314148e+01
  6.12044369e-06]]
gt: [[ 12. 381.  89. 113.]
 [233. 305.  72.  90.]
 [299. 295.  61.  92.]
 [372. 338.  47.  62.]
 [418. 319.  53.  66.]
 [396. 280.  25.  32.]
 [555. 324.  31.  47.]
 [651. 275.  36.  48.]
 [196. 287.  30.  35.]
 [  0. 308.  25.  43.]
 [757. 272.  35.  43.]
 [458. 276.  20.  28.]
 [479. 282.  22.  28.]
 [907. 282.  29.  32.]
 [929. 267.  26.  33.]
 [841. 269.  31.  31.]
 [787. 280.  30.  40.]
 [706. 272.  24.  25.]
 [369. 277.  25.  30.]]
ignore: [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Processing medium:   0%|          | 0/61 [00:00<?, ?it/s]

pred: [[8.41710449e+02 1.57113281e+02 4.50572510e+01 6.42401733e+01
  4.73558247e-01]
 [8.44086365e+02 1.58565521e+02 4.70535889e+01 6.22646637e+01
  4.29454029e-01]
 [8.44982483e+02 1.56166840e+02 4.95936890e+01 5.96423340e+01
  2.67556638e-01]
 ...
 [2.92351349e+02 3.48508209e+02 1.52330597e+02 1.64673920e+02
  1.05312094e-04]
 [1.49468033e+02 3.91960175e+02 1.10332077e+02 1.04237915e+02
  6.48155474e-05]
 [3.30434082e+02 3.48590393e+02 5.21390076e+01 5.78713684e+01
  3.26110057e-05]]
gt: [[820. 128.  74.  96.]]
ignore: [1.]
thresh: 0.5
pred_recall: [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

Processing medium:  25%|██▍       | 15/61 [00:00<00:00, 130.59it/s]

...


Processing hard:  49%|████▉     | 30/61 [00:00<00:00, 144.60it/s]

pred: [[1.38822784e+01 3.71722168e+02 8.03925400e+01 1.23177612e+02
  9.99365211e-01]
 [1.38418617e+01 3.71103943e+02 7.88164978e+01 1.21154175e+02
  9.98752832e-01]
 [1.33732185e+01 3.71722931e+02 8.01895142e+01 1.21477539e+02
  9.98703003e-01]
 ...
 [1.16287659e+02 4.88374664e+02 3.65837402e+01 4.03504944e+01
  7.86295914e-06]
 [7.06868057e+01 3.16275513e+02 1.84920120e+01 2.49289551e+01
  6.93242600e-06]
 [1.33875153e+02 4.16106262e+02 3.51509094e+01 4.47314148e+01
  6.12044369e-06]]
gt: [[ 12. 381.  89. 113.]
 [233. 305.  72.  90.]
 [299. 295.  61.  92.]
 [372. 338.  47.  62.]
 [418. 319.  53.  66.]
 [396. 280.  25.  32.]
 [555. 324.  31.  47.]
 [651. 275.  36.  48.]
 [196. 287.  30.  35.]
 [  0. 308.  25.  43.]
 [757. 272.  35.  43.]
 [458. 276.  20.  28.]
 [479. 282.  22.  28.]
 [907. 282.  29.  32.]
 [929. 267.  26.  33.]
 [841. 269.  31.  31.]
 [787. 280.  30.  40.]
 [706. 272.  24.  25.]
 [369. 277.  25.  30.]]
ignore: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

Processing hard: 100%|██████████| 61/61 [00:00<00:00, 172.13it/s]

pred: [[8.41710449e+02 1.57113281e+02 4.50572510e+01 6.42401733e+01
  4.73558247e-01]
 [8.44086365e+02 1.58565521e+02 4.70535889e+01 6.22646637e+01
  4.29454029e-01]
 [8.44982483e+02 1.56166840e+02 4.95936890e+01 5.96423340e+01
  2.67556638e-01]
 ...
 [2.92351349e+02 3.48508209e+02 1.52330597e+02 1.64673920e+02
  1.05312094e-04]
 [1.49468033e+02 3.91960175e+02 1.10332077e+02 1.04237915e+02
  6.48155474e-05]
 [3.30434082e+02 3.48590393e+02 5.21390076e+01 5.78713684e+01
  3.26110057e-05]]
gt: [[820. 128.  74.  96.]]
ignore: [1.]
thresh: 0.5
pred_recall: [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 

[0.14644062587499715, 0.1169579742695058, 0.10942067653744594]

In [22]:
evaluation(results, './widerface_evaluate/ground_truth/')

Processing medium:   0%|          | 0/61 [00:00<?, ?it/s]

pred: [[1.38822784e+01 3.71722168e+02 8.03925400e+01 1.23177612e+02
  9.99365211e-01]
 [3.96595520e+02 2.78831787e+02 2.69696045e+01 3.41580200e+01
  9.97875094e-01]
 [9.06743652e+02 2.76349426e+02 2.85372314e+01 3.57178040e+01
  9.97193098e-01]
 ...
 [6.79910156e+02 1.65503647e+02 1.18421021e+01 1.37754364e+01
  3.81631544e-05]
 [2.22176819e+02 2.96287231e+02 3.00100403e+01 4.23926697e+01
  2.46803011e-05]
 [1.01416797e+03 4.70918518e+02 1.02606201e+01 1.35795288e+01
  2.16534809e-05]]
gt: [[ 12. 381.  89. 113.]
 [233. 305.  72.  90.]
 [299. 295.  61.  92.]
 [372. 338.  47.  62.]
 [418. 319.  53.  66.]
 [396. 280.  25.  32.]
 [555. 324.  31.  47.]
 [651. 275.  36.  48.]
 [196. 287.  30.  35.]
 [  0. 308.  25.  43.]
 [757. 272.  35.  43.]
 [458. 276.  20.  28.]
 [479. 282.  22.  28.]
 [907. 282.  29.  32.]
 [929. 267.  26.  33.]
 [841. 269.  31.  31.]
 [787. 280.  30.  40.]
 [706. 272.  24.  25.]
 [369. 277.  25.  30.]]
ignore: [1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Processing hard:   0%|          | 0/61 [00:00<?, ?it/s]

ignore: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1.]
thresh: 0.5
pred_recall: [ 1.  2.  3.  4.  5.  6.  7.  8.  9.  9. 10. 11. 12. 12. 13. 13. 13. 13.
 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 13. 14. 14. 14. 14. 14. 14. 14.
 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 14. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.
 16. 16. 16. 16. 16. 16. 16. 16.

Processing hard:  49%|████▉     | 30/61 [00:00<00:00, 240.63it/s]

pred: [[1.38822784e+01 3.71722168e+02 8.03925400e+01 1.23177612e+02
  9.99365211e-01]
 [3.96595520e+02 2.78831787e+02 2.69696045e+01 3.41580200e+01
  9.97875094e-01]
 [9.06743652e+02 2.76349426e+02 2.85372314e+01 3.57178040e+01
  9.97193098e-01]
 ...
 [6.79910156e+02 1.65503647e+02 1.18421021e+01 1.37754364e+01
  3.81631544e-05]
 [2.22176819e+02 2.96287231e+02 3.00100403e+01 4.23926697e+01
  2.46803011e-05]
 [1.01416797e+03 4.70918518e+02 1.02606201e+01 1.35795288e+01
  2.16534809e-05]]
gt: [[ 12. 381.  89. 113.]
 [233. 305.  72.  90.]
 [299. 295.  61.  92.]
 [372. 338.  47.  62.]
 [418. 319.  53.  66.]
 [396. 280.  25.  32.]
 [555. 324.  31.  47.]
 [651. 275.  36.  48.]
 [196. 287.  30.  35.]
 [  0. 308.  25.  43.]
 [757. 272.  35.  43.]
 [458. 276.  20.  28.]
 [479. 282.  22.  28.]
 [907. 282.  29.  32.]
 [929. 267.  26.  33.]
 [841. 269.  31.  31.]
 [787. 280.  30.  40.]
 [706. 272.  24.  25.]
 [369. 277.  25.  30.]]
ignore: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]

Processing hard: 100%|██████████| 61/61 [00:00<00:00, 252.25it/s]


[[4.96855591e+02 1.85095779e+02 6.98511963e+01 9.62038116e+01
  9.90212560e-01]
 [2.76366364e+02 4.73786957e+02 3.00406494e+01 3.95450745e+01
  9.88846838e-01]
 [4.11274048e+02 2.64239380e+02 7.62539368e+01 6.11599121e+01
  1.99069709e-01]
 [4.00552887e+02 2.40765701e+02 9.85278931e+01 1.00281601e+02
  1.37716457e-01]
 [5.02587128e+02 8.08082031e+02 5.33730164e+01 7.62035522e+01
  1.29916698e-01]
 [2.53167160e+02 7.08812408e+01 1.35552216e+01 1.74280167e+01
  6.98350370e-02]
 [3.56072662e+02 5.27152588e+02 2.57379456e+01 3.25044556e+01
  5.16574495e-02]
 [4.93530457e+02 7.86634155e+02 8.08646240e+01 1.26279846e+02
  5.15755005e-02]
 [3.93047516e+02 2.05563705e+02 2.20391876e+02 2.96928833e+02
  4.07013968e-02]
 [4.94609375e+02 2.55920105e+02 1.10416382e+02 1.23459473e+02
  3.48999389e-02]
 [6.15924500e+02 3.98987610e+02 1.11132690e+02 1.58111267e+02
  3.39584686e-02]
 [3.58343018e+02 4.20468781e+02 4.45989380e+01 7.46746521e+01
  3.11056487e-02]
 [4.42945129e+02 2.12777985e+02 9.175750

[0.8580955458708722, 0.7860826134632832, 0.7431656772412731]

In [33]:
preds_txt = get_preds('./widerface_evaluate/widerface_txt/')


Reading Predictions : 100%|██████████| 61/61 [00:08<00:00,  6.93it/s]


In [34]:
preds_txt.keys()

dict_keys(['15--Stock_Market', '46--Jockey', '47--Matador_Bullfighter', '11--Meeting', '26--Soldier_Drilling', '39--Ice_Skating', '29--Students_Schoolkids', '57--Angler', '18--Concerts', '45--Balloonist', '24--Soldier_Firing', '31--Waiter_Waitress', '41--Swimming', '0--Parade', '40--Gymnastics', '2--Demonstration', '43--Row_Boat', '36--Football', '27--Spa', '12--Group', '34--Baseball', '6--Funeral', '51--Dresses', '10--People_Marching', '56--Voter', '42--Car_Racing', '61--Street_Battle', '9--Press_Conference', '14--Traffic', '50--Celebration_Or_Party', '38--Tennis', '48--Parachutist_Paratrooper', '5--Car_Accident', '35--Basketball', '19--Couple', '16--Award_Ceremony', '8--Election_Campain', '55--Sports_Coach_Trainer', '49--Greeting', '22--Picnic', '21--Festival', '20--Family_Group', '54--Rescue', '7--Cheering', '25--Soldier_Patrol', '32--Worker_Laborer', '28--Sports_Fan', '44--Aerobics', '30--Surgeons', '52--Photographers', '53--Raid', '3--Riot', '23--Shoppers', '1--Handshaking', '17--

In [36]:
preds_txt['15--Stock_Market']['15_Stock_Market_Stock_Market_15_483']

array([[ 6.14000000e+02,  5.67000000e+02,  2.46000000e+02,
         2.70000000e+02,  9.47911400e-01],
       [ 4.86000000e+02,  4.76000000e+02,  2.20000000e+02,
         3.46000000e+02,  2.32893240e-01],
       [ 5.24000000e+02,  4.86000000e+02,  1.35000000e+02,
         2.07000000e+02,  1.80130900e-01],
       [ 6.23000000e+02,  5.67000000e+02,  3.58000000e+02,
         4.54000000e+02,  1.51137960e-01],
       [ 5.60000000e+02,  1.50000000e+02,  3.99000000e+02,
         5.08000000e+02,  1.19611934e-01],
       [ 4.93000000e+02,  3.79000000e+02,  3.96000000e+02,
         4.72000000e+02,  1.14329670e-01],
       [ 3.66000000e+02,  7.29000000e+02,  5.10000000e+01,
         6.60000000e+01,  9.68831550e-02],
       [ 4.10000000e+02,  7.80000000e+02,  3.85000000e+02,
         5.51000000e+02,  7.98499360e-02],
       [ 3.41000000e+02,  3.97000000e+02,  3.13000000e+02,
         4.54000000e+02,  7.16069800e-02],
       [ 3.92000000e+02,  5.34000000e+02,  4.09000000e+02,
         4.51000000e+02

In [46]:
preds_txt['15--Stock_Market']['15_Stock_Market_Stock_Market_15_483'].dtype

dtype('float64')

In [49]:
results['15--Stock_Market']['15_Stock_Market_Stock_Market_15_483']

array([[ 6.14678711e+02,  5.67699707e+02,  2.45845154e+02,
         2.69710327e+02,  9.47911382e-01],
       [ 4.86948212e+02,  4.76519470e+02,  2.19609161e+02,
         3.45916199e+02,  2.32893243e-01],
       [ 5.24796448e+02,  4.86052429e+02,  1.34648804e+02,
         2.07618896e+02,  1.80130899e-01],
       [ 6.23333984e+02,  5.67045715e+02,  3.58484375e+02,
         4.54404541e+02,  1.51137963e-01],
       [ 5.60474854e+02,  1.50883331e+02,  3.98898132e+02,
         5.07254242e+02,  1.19611934e-01],
       [ 4.93502930e+02,  3.79408783e+02,  3.95729248e+02,
         4.72161957e+02,  1.14329673e-01],
       [ 3.66979095e+02,  7.29027161e+02,  5.07568665e+01,
         6.62018433e+01,  9.68831554e-02],
       [ 4.10173828e+02,  7.80479553e+02,  3.85808838e+02,
         5.50782043e+02,  7.98499361e-02],
       [ 3.41533875e+02,  3.97497162e+02,  3.12571350e+02,
         4.54405426e+02,  7.16069788e-02],
       [ 3.92213684e+02,  5.34876465e+02,  4.09073486e+02,
         4.50545410e+02

In [33]:
!ls ./widerface_evaluate/widerface_txt//24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.txt

./widerface_evaluate/widerface_txt//24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.txt


In [36]:
'/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg'.split('/')

['', '24--Soldier_Firing', '24_Soldier_Firing_Soldier_Firing_24_329.jpg']

In [37]:
'/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg'.split('/')[2][:-4]

'24_Soldier_Firing_Soldier_Firing_24_329'

In [42]:
results = defaultdict(dict)
results

defaultdict(dict, {})

In [43]:
results['a']

{}

In [44]:
tmp = defaultdict(list)
results = defaultdict(tmp)

TypeError: first argument must be callable or None

In [55]:
from scipy.io import loadmat

In [56]:
gt_mat = loadmat(os.path.join('./widerface_evaluate/ground_truth/', 'wider_face_val.mat'))

In [59]:
gt_mat['file_list'].shape

(61, 1)

In [65]:
gt_mat['file_list'][0][0]

array([[array(['0_Parade_marchingband_1_465'], dtype='<U27')],
       [array(['0_Parade_Parade_0_628'], dtype='<U21')],
       [array(['0_Parade_marchingband_1_765'], dtype='<U27')],
       [array(['0_Parade_Parade_0_194'], dtype='<U21')],
       [array(['0_Parade_marchingband_1_379'], dtype='<U27')],
       [array(['0_Parade_Parade_0_814'], dtype='<U21')],
       [array(['0_Parade_Parade_0_470'], dtype='<U21')],
       [array(['0_Parade_marchingband_1_1045'], dtype='<U28')],
       [array(['0_Parade_marchingband_1_556'], dtype='<U27')],
       [array(['0_Parade_Parade_0_829'], dtype='<U21')],
       [array(['0_Parade_marchingband_1_593'], dtype='<U27')],
       [array(['0_Parade_Parade_0_29'], dtype='<U20')],
       [array(['0_Parade_Parade_0_72'], dtype='<U20')],
       [array(['0_Parade_Parade_0_205'], dtype='<U21')],
       [array(['0_Parade_marchingband_1_881'], dtype='<U27')],
       [array(['0_Parade_marchingband_1_490'], dtype='<U27')],
       [array(['0_Parade_Parade_0_125'], 

In [77]:
hard_mat = loadmat(os.path.join('./widerface_evaluate/ground_truth/', 'wider_hard_val.mat'))
easy_mat = loadmat(os.path.join('./widerface_evaluate/ground_truth/', 'wider_easy_val.mat'))
medium_mat = loadmat(os.path.join('./widerface_evaluate/ground_truth/', 'wider_medium_val.mat'))


In [79]:
hard_mat['gt_list'][0][0][1]

array([array([[ 1],
       [ 2],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [21],
       [22],
       [23],
       [24],
       [25]], dtype=uint8)], dtype=object)

In [80]:
medium_mat['gt_list'][0][0][1]

array([array([[13],
       [15],
       [16]], dtype=uint8)], dtype=object)

In [81]:
easy_mat['gt_list'][0][0][1]

array([array([], shape=(0, 1), dtype=uint8)], dtype=object)

In [89]:
hard_mat['gt_list'][0][0].shape

(115, 1)

In [92]:
hard_mat['gt_list'][0][0][11]

array([array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19],
       [20],
       [22],
       [23],
       [24],
       [25],
       [26]], dtype=uint8)], dtype=object)

In [93]:
medium_mat['gt_list'][0][0][11]

array([array([[ 1],
       [ 2],
       [ 4],
       [ 8],
       [17]], dtype=uint8)], dtype=object)

In [94]:
easy_mat['gt_list'][0][0][11]

array([array([[1],
       [2]], dtype=uint8)], dtype=object)

In [96]:
!ls data/widerface/val

images	wider_val.txt


In [97]:
!head -n 10 data/widerface/val/wider_val.txt

/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_10.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_1037.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_115.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_129.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_133.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_15.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_254.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_264.jpg
/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_268.jpg


In [127]:
with open('data/widerface/val/wider_val.txt', 'r') as f:
    val_list = f.read().split()

In [128]:
len(val_list)

3226

In [129]:
val_list[:2]

['/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_329.jpg',
 '/24--Soldier_Firing/24_Soldier_Firing_Soldier_Firing_24_10.jpg']

In [103]:
from sklearn.utils import shuffle

In [105]:
val_list_half = shuffle(val_list)[:1600]

In [130]:
val_list_half[:10]

['/30--Surgeons/30_Surgeons_Surgeons_30_819.jpg\n',
 '/61--Street_Battle/61_Street_Battle_streetfight_61_395.jpg\n',
 '/56--Voter/56_Voter_peoplevoting_56_122.jpg\n',
 '/21--Festival/21_Festival_Festival_21_462.jpg\n',
 '/52--Photographers/52_Photographers_photographertakingphoto_52_125.jpg\n',
 '/2--Demonstration/2_Demonstration_Political_Rally_2_137.jpg\n',
 '/35--Basketball/35_Basketball_basketballgame_ball_35_681.jpg\n',
 '/35--Basketball/35_Basketball_basketballgame_ball_35_827.jpg\n',
 '/35--Basketball/35_Basketball_playingbasketball_35_251.jpg\n',
 '/2--Demonstration/2_Demonstration_Protesters_2_352.jpg\n']

In [107]:
with open('data/widerface/val/wider_val_half.txt', 'w+') as f:
    for line in val_list_half:
        f.write(line)

In [131]:
with open('data/widerface/val/wider_val_half.txt', 'r') as f:
    val_list_half = f.read().split()

In [132]:
val_list_half[:10]

['/30--Surgeons/30_Surgeons_Surgeons_30_819.jpg',
 '/61--Street_Battle/61_Street_Battle_streetfight_61_395.jpg',
 '/56--Voter/56_Voter_peoplevoting_56_122.jpg',
 '/21--Festival/21_Festival_Festival_21_462.jpg',
 '/52--Photographers/52_Photographers_photographertakingphoto_52_125.jpg',
 '/2--Demonstration/2_Demonstration_Political_Rally_2_137.jpg',
 '/35--Basketball/35_Basketball_basketballgame_ball_35_681.jpg',
 '/35--Basketball/35_Basketball_basketballgame_ball_35_827.jpg',
 '/35--Basketball/35_Basketball_playingbasketball_35_251.jpg',
 '/2--Demonstration/2_Demonstration_Protesters_2_352.jpg']

# multi processing

In [136]:
from collections import defaultdict

def predict_single(img_file_name):
    torch.set_grad_enabled(False)

    cfg = None
    if args.network == "mobile0.25":
        cfg = cfg_mnet
    elif args.network == "resnet50":
        cfg = cfg_re50
    # net and model
    net = RetinaFace(cfg=cfg, phase = 'test')
    net = load_model(net, args.trained_model, args.cpu)
    net.eval()
    print('Finished loading model!')
    #print(net)
    cudnn.benchmark = True
    device = torch.device("cpu" if args.cpu else "cuda")
    net = net.to(device)

    # testing dataset
    testset_folder = args.dataset_folder
    #testset_list = args.dataset_folder[:-7] + "wider_val.txt"
    #testset_list = args.dataset_folder[:-7] + "wider_val_half.txt"

    #with open(testset_list, 'r') as fr:
    #    test_dataset = fr.read().split()
    num_images = 1 #len(test_dataset)

    _t = {'forward_pass': Timer(), 'misc': Timer()}

    # testing begin
    results = defaultdict(dict)
    #for i, img_name in enumerate(test_dataset):
    if True:
        img_name = img_file_name
        image_path = testset_folder + img_name
        img_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
        img = np.float32(img_raw)

        # testing scale
        target_size = 1600
        max_size = 2150
        im_shape = img.shape
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])
        resize = float(target_size) / float(im_size_min)
        # prevent bigger axis from being more than max_size:
        if np.round(resize * im_size_max) > max_size:
            resize = float(max_size) / float(im_size_max)
        if args.origin_size:
            resize = 1

        if resize != 1:
            img = cv2.resize(img, None, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)
        im_height, im_width, _ = img.shape
        scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
        img -= (104, 117, 123)
        img = img.transpose(2, 0, 1)
        img = torch.from_numpy(img).unsqueeze(0)
        img = img.to(device)
        scale = scale.to(device)

        _t['forward_pass'].tic()
        loc, conf, landms = net(img)  # forward pass
        _t['forward_pass'].toc()
        _t['misc'].tic()
        priorbox = PriorBox(cfg, image_size=(im_height, im_width))
        priors = priorbox.forward()
        priors = priors.to(device)
        prior_data = priors.data
        boxes = decode(loc.data.squeeze(0), prior_data, cfg['variance'])
        boxes = boxes * scale / resize
        boxes = boxes.cpu().numpy()
        scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
        landms = decode_landm(landms.data.squeeze(0), prior_data, cfg['variance'])
        scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2]])
        scale1 = scale1.to(device)
        landms = landms * scale1 / resize
        landms = landms.cpu().numpy()

        # ignore low scores
        inds = np.where(scores > args.confidence_threshold)[0]
        boxes = boxes[inds]
        landms = landms[inds]
        scores = scores[inds]

        # keep top-K before NMS
        order = scores.argsort()[::-1]
        # order = scores.argsort()[::-1][:args.top_k]
        boxes = boxes[order]
        landms = landms[order]
        scores = scores[order]

        # do NMS
        dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
        keep = py_cpu_nms(dets, args.nms_threshold)
        # keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
        dets = dets[keep, :]
        landms = landms[keep]

        # keep top-K faster NMS
        # dets = dets[:args.keep_top_k, :]
        # landms = landms[:args.keep_top_k, :]

        #dets = np.concatenate((dets, landms), axis=1)
        _t['misc'].toc()
        
        event = img_name.split('/')[1]
        event_img = img_name.split('/')[2][:-4]

        #if event_img not in results[event]:
        dets[:, 2] = dets[:, 2] - dets[:, 0]
        dets[:, 3] = dets[:, 3] - dets[:, 1]
        #results[event][event_img] = dets[:, :5]
        #print('results:', results)
    return event, event_img, dets
            

        

In [137]:
%%time

predict_single('/30--Surgeons/30_Surgeons_Surgeons_30_819.jpg')

Loading pretrained model from ./weights/mobilenet0.25_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:300
Finished loading model!
CPU times: user 4.46 s, sys: 1.33 s, total: 5.79 s
Wall time: 315 ms


('30--Surgeons',
 '30_Surgeons_Surgeons_30_819',
 array([[3.6067139e+02, 2.5855682e+01, 5.3501404e+01, 7.1234787e+01,
         9.9984026e-01],
        [4.6462750e+02, 2.2963598e+01, 4.7657654e+01, 6.2339951e+01,
         9.9912089e-01],
        [5.7824902e+02, 4.2127449e+01, 5.3097534e+01, 6.9189972e+01,
         9.9900025e-01],
        [8.2206567e+02, 4.2721260e+01, 3.9951782e+01, 4.9774292e+01,
         2.1437963e-02]], dtype=float32))

In [138]:
from multiprocessing import Pool

#def pred_single(fn):
#    return ('result', fn.strip())

def parallel_pred(fn_list):
    pool = Pool(4)
    res = pool.map(predict_single, fn_list)
    pool.close()
    pool.join()
    return res

In [141]:
torch.multiprocessing.set_start_method('spawn')

RuntimeError: context has already been set

In [140]:
%%time 

tmp = parallel_pred(val_list_half[:4])

Loading pretrained model from ./weights/mobilenet0.25_Final.pth
Loading pretrained model from ./weights/mobilenet0.25_Final.pth
Loading pretrained model from ./weights/mobilenet0.25_Final.pth
Loading pretrained model from ./weights/mobilenet0.25_Final.pth


RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method

In [123]:
tmp

[('result', '/30--Surgeons/30_Surgeons_Surgeons_30_819.jpg'),
 ('result', '/61--Street_Battle/61_Street_Battle_streetfight_61_395.jpg'),
 ('result', '/56--Voter/56_Voter_peoplevoting_56_122.jpg'),
 ('result', '/21--Festival/21_Festival_Festival_21_462.jpg'),
 ('result',
  '/52--Photographers/52_Photographers_photographertakingphoto_52_125.jpg'),
 ('result', '/2--Demonstration/2_Demonstration_Political_Rally_2_137.jpg'),
 ('result', '/35--Basketball/35_Basketball_basketballgame_ball_35_681.jpg'),
 ('result', '/35--Basketball/35_Basketball_basketballgame_ball_35_827.jpg'),
 ('result', '/35--Basketball/35_Basketball_playingbasketball_35_251.jpg'),
 ('result', '/2--Demonstration/2_Demonstration_Protesters_2_352.jpg'),
 ('result',
  '/2--Demonstration/2_Demonstration_Demonstration_Or_Protest_2_117.jpg'),
 ('result', '/30--Surgeons/30_Surgeons_Surgeons_30_77.jpg')]